In [1]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

import matlab.engine as engi
import matlab as mat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from pyearth import Earth

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt

In [2]:
def load_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    commit_guru_df = pd.read_csv(commit_guru_path)
    cols = understand_df.columns.tolist()
    
    commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
                                                   'author_email','fileschanged','author_date',
                                                   'author_date_unix_timestamp', 'commit_message',
                                                  'classification', 'fix', 'contains_bug','fixes'],axis=1)

    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    df = understand_df.merge(commit_guru_df,on='commit_hash')
    cols = df.columns.tolist()
    cols = cols[1:] + [cols[0]]
    df = df[cols]
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.to_csv('data/converted/'+ project + '_understand.csv',index=False)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    return X,y

def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

In [3]:
def run_self(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test.CountLineCode
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf =  RandomForestClassifier()
    clf.fit(X_train,y_train)
    importance = 0
    predicted = clf.predict(X_test)
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [4]:
def run_self_CFS(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test.CountLineCode
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    df_smote,cols = apply_cfs(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = RandomForestClassifier()
    clf.fit(X_train,y_train)
    importance = clf.feature_importances_
    predicted = clf.predict(X_test[cols[:-1]])
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [5]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

In [143]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
pci_20_list = {}
ifa_list = {}
featue_importance = {}
for project in projects:
    try:
        if project == '.DS_Store':
            continue
#         if project != 'org.alloytools.alloy':
#             continue
        if project != 'jfairy':
            continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance = run_self(project)
        recall_list[project] = recall
        precision_list[project] = precision
        pf_list[project] = pf
        f1_list[project] = f1
        g_list[project] = g_score
        auc_list[project] = auc
        pci_20_list[project] = pci_20
        ifa_list[project] = ifa
        featue_importance[project] = importance
    except Exception as e:
        print(e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
final_result['pci_20'] = pci_20_list
final_result['ifa'] = ifa_list
with open('results/Performance/process+product_nb.pkl', 'wb') as handle:
    pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

+++++++++++++++++   jfairy  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.72      0.68      0.70        82
           1       0.80      0.83      0.82       128

    accuracy                           0.77       210
   macro avg       0.76      0.76      0.76       210
weighted avg       0.77      0.77      0.77       210



In [6]:
project = 'guice'
commit_guru_path = 'data/commit_guru/' + project + '.csv'
commit_guru_file_path = 'data/commit_guru_file_level/' + project + '_file.csv'
understand_path = 'data/understand_files_all/' + project + '_understand.csv'
df_1 = pd.read_csv(commit_guru_path)
df_2 = pd.read_csv(commit_guru_file_path)
df_3 = pd.read_csv(understand_path)
df_3 = df_3.dropna(axis = 1,how='all')
df_4 = df_2.merge(df_1,on='commit_hash')
df_3['Name'] = df_3.Name.str.rsplit('.',1).str[1]

In [32]:
df_2 = df_2[df_2['file_name'].str.contains('.java')]
df_4 = df_2.reset_index(drop = True)
print(df_4.loc[0])

commit_hash         66b415a2066cac9f36ed58070777de388f63a3a4"
file_name      src/com/google/inject/ConstructionContext.java
la                                                        124
ld                                                          0
lt                                                          0
age                                                         0
ndev                                                        1
nuc                                                         0
ns                                                          1
exp                                                        41
sexp                                                        3
rexp                                                        0
nd                                                          1
Name: 0, dtype: object


In [37]:
df_4['file_name'] = df_4.file_name.str.rsplit('/',1).str[1].str.split('.').str[0]

In [40]:
df_4

,commit_hash,file_name,la,ld,lt,age,ndev,nuc,ns,exp,sexp,rexp,nd
0,"66b415a2066cac9f36ed58070777de388f63a3a4""",ConstructionContext,124,0,0,0.000000,1,0,1,41,3.0,0.000000,1.0
1,"66b415a2066cac9f36ed58070777de388f63a3a4""",Container,103,0,0,0.000000,1,0,1,42,1.0,0.000000,1.0
2,"66b415a2066cac9f36ed58070777de388f63a3a4""",ContainerBuilder,483,0,0,0.000000,1,0,1,43,2.0,0.000000,1.0
3,"66b415a2066cac9f36ed58070777de388f63a3a4""",ContainerImpl,556,0,0,0.000000,1,0,1,44,3.0,0.000000,1.0
4,"66b415a2066cac9f36ed58070777de388f63a3a4""",Context,57,0,0,0.000000,1,0,1,45,4.0,0.000000,1.0
5,"66b415a2066cac9f36ed58070777de388f63a3a4""",DependencyException,37,0,0,0.000000,1,0,1,46,5.0,0.000000,1.0
6,"66b415a2066cac9f36ed58070777de388f63a3a4""",ExternalContext,73,0,0,0.000000,1,0,1,47,6.0,0.000000,1.0
7,"66b415a2066cac9f36ed58070777de388f63a3a4""",Factory,34,0,0,0.000000,1,0,1,48,7.0,0.000000,1.0
8,"66b415a2066cac9f36ed58070777de388f63a3a4""",Inject,46,0,0,0.000000,1,0,1,49,8.0,0.000000,1.0
9,"66b415a2066cac9f36ed58070777de388f63a3a4""",InternalContext,80,0,0,0.000000,1,0,1,50,9.0,0.000000,1.0


In [128]:
df_4.columns = ['commit_hash', 'Name', 'file_la', 'file_ld', 'file_lt', 'file_age',
       'file_ddev', 'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm',
       'file_adev', 'file_nadev', 'file_avg_nddev', 'file_avg_nadev',
       'file_avg_ncomm', 'file_ns', 'file_exp', 'file_sexp', 'file_rexp',
       'file_nd', 'file_sctr', 'age', 'author_date',
       'author_date_unix_timestamp', 'author_email', 'author_name',
       'classification', 'commit_message', 'entrophy', 'exp', 'fileschanged',
       'fix', 'la', 'ld', 'lt', 'nd', 'ndev', 'nf', 'ns', 'nuc',
       'parent_hashes', 'rexp', 'sexp', 'contains_bug', 'fixes']

In [129]:
df_5 = df_4.merge(df_3,on=['commit_hash','Name'])

In [130]:
df_5 = df_5.drop(labels = ['parent_hashes','author_name','author_name',
            'author_email','fileschanged','author_date',
            'author_date_unix_timestamp', 'commit_message',
            'classification', 'fix', 'contains_bug','fixes',
            'commit_hash','Name','Kind'],axis=1)

In [131]:
df_5

,file_la,file_ld,file_lt,file_age,file_ddev,file_nuc,own,minor,file_ndev,file_ncomm,...,MaxInheritanceTree,MaxNesting,PercentLackOfCohesion,PercentLackOfCohesionModified,RatioCommentToCode,SumCyclomatic,SumCyclomaticModified,SumCyclomaticStrict,SumEssential,Bugs
0,13,4,64,0.003924,2,6,1.000000,0,0,0,...,1.0,2,50.0,0.0,0.31,10,10,10,6,0
1,51,0,0,0.000000,1,0,1.000000,0,0,0,...,1.0,1,50.0,50.0,0.27,6,6,6,5,1
2,15,11,73,0.054306,2,7,1.000000,0,19,27,...,1.0,2,42.0,0.0,0.29,11,11,11,7,1
3,39,0,0,0.000000,1,0,1.000000,0,0,0,...,2.0,1,41.0,41.0,0.21,5,5,5,4,1
4,5,5,25,0.058229,2,3,1.000000,0,14,25,...,1.0,0,0.0,0.0,0.67,1,1,1,1,1
5,35,5,28,0.058229,2,3,1.000000,0,14,27,...,2.0,2,44.0,44.0,0.12,8,8,8,6,1
6,25,2,51,3.063333,2,4,0.510000,0,23,88,...,1.0,1,62.0,56.0,0.25,9,9,9,8,1
7,25,2,51,3.063333,2,4,0.510000,0,23,88,...,1.0,1,62.0,56.0,0.25,9,9,9,8,0
8,5,0,81,3.063333,2,11,0.651316,0,41,120,...,1.0,2,37.0,0.0,0.29,12,12,12,8,1
9,5,0,81,3.063333,2,11,0.651316,0,41,120,...,1.0,2,37.0,0.0,0.29,12,12,12,8,0


In [134]:
print(df_5.shape)
df_5.dropna(inplace=True)
df_5.reset_index(drop=True,inplace=True)
y = df_5.Bugs
X = df_5.drop('Bugs',axis = 1)
print(df_5.shape)

(895, 80)
(895, 80)


In [135]:
cols = X.columns
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X,columns = cols)

In [150]:
def run_self_CFS(X,y):
#     X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test.CountLineCode
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    df_smote,cols = apply_cfs(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = GaussianNB()
    clf.fit(X_train,y_train)
    importance = 0
    predicted = clf.predict(X_test[cols[:-1]])
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [151]:
run_self_CFS(X,y)

              precision    recall  f1-score   support

           0       0.37      0.06      0.11       113
           1       0.69      0.95      0.80       245

    accuracy                           0.67       358
   macro avg       0.53      0.51      0.45       358
weighted avg       0.59      0.67      0.58       358



(0.95, 0.69, 0.94, 0.8, 0.24, 0.5064836554090663, 0.48, 2, 0)

In [120]:
df_5.to_csv('tst.csv')

In [17]:
import pandas as pd

#create sample data
data = {'model': ['Lisa', 'Lisa 2', 'Macintosh 128K', 'Macintosh 512K'],
        'launched': [1983,1984,1984,1984],
        'discontinued': [1986, 1985, 1984, 1986]}

df = pd.DataFrame(data, columns = ['model', 'launched', 'discontinued'])
df

,model,launched,discontinued
0,Lisa,1983,1986
1,Lisa 2,1984,1985
2,Macintosh 128K,1984,1984
3,Macintosh 512K,1984,1986


In [20]:
df[df['model'].str.contains('sa')]

,model,launched,discontinued
0,Lisa,1983,1986
1,Lisa 2,1984,1985
